In [1]:
import os
import pandas as pd
import janitor
import numpy as np
pd.set_option('display.max_columns', 100)

DATAPATH = "../../adult-data"        

In [2]:
# (fold cell) Load web browsing data
usecols_web_desktop = ["caseid", "category", "private_domain", "visit_duration", "visit_time_local"]
usecols = ["caseid", "category", "private_domain", "page_duration", "session_start_time"]

df = (pd.concat([
        # Get web_mobile
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_mobile_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ), 
        # Get web_desktop
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_desktop_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web
        (pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_2022-06-01_2022-06-30.csv"),
            usecols=usecols,
            low_memory=False,
        )
         # Renaming columns to be consistent w/ web_mobile & web_desktop
         .rename_column("session_start_time", "visit_time_local")
         .rename_column("page_duration", "visit_duration")
        )
        ])
      .assign(caseid=lambda df: df.caseid.astype(object))      
     )
print("df of web browses")
print(f"{len(df)=:,}")
df.head()

df of web browses
len(df)=6,297,382


,caseid,private_domain,category,visit_time_local,visit_duration
0,205323077,google.com,Search Engines and Portals,2022-05-31 23:52:37,2
1,205323077,coupons.com,"Business, Shopping",2022-06-01 01:07:35,457
2,205323077,google.com,Business,2022-06-01 01:15:12,55
3,205323077,coupons.com,"Business, Shopping",2022-06-01 01:16:07,2225
4,205323077,google.com,Search Engines and Portals,2022-06-01 04:38:10,10


In [3]:
df_caseid = (df
             .rename_column("visit_duration", "seconds")
             .groupby("caseid")["seconds"].sum().reset_index()
             .sort_values("seconds", ignore_index=True)
             .assign(
                 mins=lambda df: df.seconds/60,
                 hours=lambda df: df.mins/60,
             )
            )
df_caseid.head(50)

,caseid,seconds,mins,hours
0,206433705,5,0.083333,0.001389
1,301913121,6,0.100000,0.001667
2,80623593,8,0.133333,0.002222
3,8538267,16,0.266667,0.004444
4,286591993,36,0.600000,0.010000
5,307997169,38,0.633333,0.010556
6,292286781,40,0.666667,0.011111
7,320316041,62,1.033333,0.017222
8,289407473,74,1.233333,0.020556
9,317420377,92,1.533333,0.025556


In [4]:
len(df_caseid.query("hours<1"))

63

In [5]:
len(df_caseid.query("hours<2"))

107

In [6]:
len(df_caseid.query("hours<5"))

189